In [3]:
!pip install imbalanced-learn


In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from imblearn.over_sampling import SMOTE


In [5]:
url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
X = df.drop("Class", axis=1)
y = df["Class"]

smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)

balanced_df = pd.concat([X_bal, y_bal], axis=1)
balanced_df["Class"].value_counts()


,count
Class,
0,763
1,763


In [7]:
def sampling1(data):
    return data.sample(frac=0.8, random_state=1)

def sampling2(data):
    return data.sample(frac=0.6, random_state=2)

def sampling3(data):
    return data.sample(frac=0.7, random_state=3)

def sampling4(data):
    return data.sample(frac=0.5, random_state=4)

def sampling5(data):
    return data.sample(frac=0.9, random_state=5)


In [8]:
models = {
    "M1": LogisticRegression(max_iter=1000),
    "M2": DecisionTreeClassifier(),
    "M3": RandomForestClassifier(),
    "M4": KNeighborsClassifier(),
    "M5": GaussianNB()
}


In [9]:
sampling_methods = {
    "Sampling1": sampling1,
    "Sampling2": sampling2,
    "Sampling3": sampling3,
    "Sampling4": sampling4,
    "Sampling5": sampling5
}

results = pd.DataFrame()

for s_name, s_func in sampling_methods.items():

    sample_data = s_func(balanced_df)

    X_sample = sample_data.drop("Class", axis=1)
    y_sample = sample_data["Class"]

    X_train, X_test, y_train, y_test = train_test_split(
        X_sample, y_sample, test_size=0.3, random_state=42
    )

    accuracies = []

    for m_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred) * 100
        accuracies.append(acc)

    results[s_name] = accuracies

results.index = models.keys()
results


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1,92.915531,90.545455,90.965732,93.449782,94.417476
M2,98.092643,96.363636,98.130841,97.379913,97.330097
M3,99.182561,98.181818,99.376947,99.126638,99.757282
M4,82.288828,80.727273,82.554517,82.969432,85.436893
M5,88.828338,82.181818,81.619938,75.545852,86.650485


In [11]:
best_sampling = results.idxmax(axis=1)
best_accuracy = results.max(axis=1)

final_result = pd.DataFrame({
    "Best Sampling Technique": best_sampling,
    "Highest Accuracy (%)": best_accuracy
})

print("\nBest Sampling for Each Model:")
final_result



Best Sampling for Each Model:


,Best Sampling Technique,Highest Accuracy (%)
M1,Sampling5,94.417476
M2,Sampling1,97.820163
M3,Sampling5,99.757282
M4,Sampling5,85.436893
M5,Sampling1,88.828338
